In [1]:
import pandas as pd
from connection import connect
from utils.model_loader import ModelRegistry
from utils.translate_language import convert_language


In [4]:
#connection
co_oltp,etl_conn,etl_conn_or=connect()

In [ ]:
# Carga de modelos
registry = ModelRegistry()
registry.preload_model('en', 'es')
registry.preload_model('en', 'fr')
tokenizer_es, model_es = registry.get_model('en', 'es')
tokenizer_fr, model_fr = registry.get_model('en', 'fr')

## Extract

In [5]:
# extract DimProductCategory

dim_product_subcategory=pd.read_sql_table('product_subcategory',co_oltp,schema='production')

## Transform


In [6]:
dim_product_subcategory=dim_product_subcategory.rename(columns={'product_subcategory_id':'product_subcategory_alternate_key','name':'english_product_subcategory_name'})

dim_product_subcategory=dim_product_subcategory.drop(['rowguid','modified_date'],axis=1)


dim_product_subcategory=convert_language('english_product_subcategory_name','spanish_product_subcategory_name',tokenizer_es, model_es, dim_product_subcategory)
dim_product_subcategory=convert_language('english_product_subcategory_name','french_product_subcategory_name', tokenizer_fr, model_fr, dim_product_subcategory)

dim_product_category=pd.read_sql_table('dim_product_category',etl_conn)

dim_product_subcategory = dim_product_subcategory.merge(
    dim_product_category[['product_category_alternate_key', 'product_category_key']],
    left_on='product_category_id',
    right_on='product_category_alternate_key',
    how='left'
)

dim_product_subcategory.drop(['product_category_alternate_key','product_category_id'], inplace=True,axis=1)

dim_product_subcategory

--- Loading model Helsinki-NLP/opus-mt-en-es (this should only happen once) ---


c:\Users\andres\Desktop\SEPTIMO SEMESTRE\DATA SCIENCE\etlexample\proyectoetl\.venv\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Found 37 total rows, but only 37 unique values to translate.
Translation complete.
--- Loading model Helsinki-NLP/opus-mt-en-fr (this should only happen once) ---


c:\Users\andres\Desktop\SEPTIMO SEMESTRE\DATA SCIENCE\etlexample\proyectoetl\.venv\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Found 37 total rows, but only 37 unique values to translate.
Translation complete.


,product_subcategory_alternate_key,english_product_subcategory_name,spanish_product_subcategory_name,french_product_subcategory_name,product_category_key
0,1,Mountain Bikes,Bicicletas de montaña,Vélos de montagne,1
1,2,Road Bikes,Bicicletas de carretera,Vélos de route,1
2,3,Touring Bikes,Bicicletas de turismo,Vélos de tourisme,1
3,4,Handlebars,Manillar,Poignées,2
4,5,Bottom Brackets,Brackets inferiores,Barres de fond,2
5,6,Brakes,Frenos,Freins,2
6,7,Chains,Cadenas,Chaînes,2
7,8,Cranksets,Cranksets,Cranksets,2
8,9,Derailleurs,Derailleurs,Dérailleurs,2
9,10,Forks,Horquillas,Fourches,2


## Load

In [ ]:
dim_product_subcategory.to_sql('dim_product_subcategory', etl_conn, if_exists='append',index=False)

37